In [ ]:
from Src.pre_processing import load_data, split, to_tensor, to_labels 
from Src.Sentan_Model import Sentan_simple
from Src.Dias_Model import Dias_Model
from Src.pre_processing import split, to_tensor

from sklearn.ensemble import RandomForestClassifier
import torch

from sklearn.model_selection import train_test_split
from torch.optim import Adam
from torch import nn

from torchmetrics import F1Score, Recall, Precision, Accuracy
import wandb


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### kfold training

In [ ]:
def train_kfold(X, y, n_folds):

    f1 = F1Score(num_classes=8)
    recall = Recall(average='macro', num_classes=8)
    precision = Precision(average='macro', num_classes=8)
    accuracy = Accuracy()

    # K-folds
    for train_idx, test_idx in split(X, y, splits=n_folds):

        # Activación del entorno
        wandb.init(project="Clasificador de sentimientos", entity="panas")

        # Fold Data
        x_train = X[train_idx]
        y_train = y[train_idx]

        x_test = X[test_idx]
        y_test = y[test_idx]

        y_test = torch.tensor(y_test)


        with wandb.init() as run:
            config = wandb.config
    
            classifier = RandomForestClassifier(n_estimators = 1000, random_state = 0)   
            pred = classifier.fit(x_train, y_train).predict(x_test)         
            
            # Login de resultados a Weights and biases
            pred = torch.tensor(pred)
            wandb.log({'Validation Accuracy': accuracy(pred, y_test), 'F1 Score': f1(
                pred, y_test), 'Recall': recall(pred, y_test), 'Precision': precision(pred, y_test)})

            print({'Validation Accuracy': accuracy(pred, y_test), 'F1 Score': f1(
                pred, y_test), 'Recall': recall(pred, y_test), 'Precision': precision(pred, y_test)})

## Data

In [ ]:
data_path = 'Data/data.pkl'

X, y, labels = load_data(data_path)

## Configuración W&B

In [ ]:
epochs = 200
learning_rate = 1e-3
batch_size = X.shape[0]


# Configuración de entorno weights and biases
wandb.config = {
    "model-name": 'Tree Model',
    "learning_rate": learning_rate,
    "epochs": epochs,
    "batch_size": batch_size

}

## Training KFold

In [ ]:
# Entrenamiento de n modelos
folds = 10

train_kfold(X, y, folds)